# Study – Glass – Stretcher: bissection

In [2]:
import pandas as pd
import tensorflow as tf

from milp import codify_network
from teste import get_minimal_explanation

2023-07-04 15:35:15.830263: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-04 15:35:36.374308: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-04 15:35:36.374328: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-07-04 15:35:38.040527: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-04 15:36:03.996573: W tensorflow/stream_executor/platform/de

In [3]:
# For type annotations
import numpy as np

In [4]:
dataset_name = 'glass'

training_data = pd.read_csv(f'datasets/{dataset_name}/train.csv')
testing_data = pd.read_csv(f'datasets/{dataset_name}/test.csv')

dataframe = pd.concat([training_data, testing_data])

keras_model = tf.keras.models.load_model(f'datasets/{dataset_name}/model_2layers_{dataset_name}.h5')

data = dataframe.to_numpy()
n_classes = dataframe['target'].nunique()

2023-07-04 15:36:32.370467: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-07-04 15:36:32.381935: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-04 15:36:32.382008: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tiago-lab-pc): /proc/driver/nvidia/version does not exist
2023-07-04 15:36:32.416663: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
mp_model, output_bounds = codify_network(keras_model, dataframe, 'fischetti', relax_constraints=False)

### Printing What the Model Predicted

_Aka_ printing the network output.

In [6]:
i = 0
print('i =', i)
network_input = data[i, :-1]
network_input = tf.reshape(tf.constant(network_input), [1, -1])
network_output = keras_model.predict(tf.constant(network_input))[0]
network_output = tf.argmax(network_output)

predictions = keras_model.predict(tf.constant(network_input))[0, 0]

print(f'Predictions: (ndarray[ndarray[{type(predictions)}]])', predictions)
classification: np.int64 = network_output.numpy()
print(f'Network output: ({type(classification)})', classification)

i = 0
1/1 [==============================] - 0s 34ms/step
Predictions: (ndarray[ndarray[<class 'numpy.float32'>]]) 0.56761974
Network output: (<class 'numpy.int64'>) 0


### Printing the Minimal Explanation

Minimal eplanations only indicates which inputs are relevant to get to a conclusion.

**Note:** The explanation happens _after_ the keras_model make its predictions.

In [7]:
mdl_aux = mp_model.clone()

minimal_explanation = get_minimal_explanation(mdl_aux, network_input, network_output, n_classes, 'fischetti', output_bounds)
minimal_explanation

[docplex.mp.LinearConstraint[input1](x_0,EQ,-0.2792306037275416),
 docplex.mp.LinearConstraint[input2](x_1,EQ,-0.267930022273471),
 docplex.mp.LinearConstraint[input3](x_2,EQ,0.6074173914551261),
 docplex.mp.LinearConstraint[input4](x_3,EQ,-0.8032068868488967),
 docplex.mp.LinearConstraint[input5](x_4,EQ,0.8170760979829164),
 docplex.mp.LinearConstraint[input6](x_5,EQ,0.0779031564307683),
 docplex.mp.LinearConstraint[input7](x_6,EQ,-0.4927117463954317),
 docplex.mp.LinearConstraint[input8](x_7,EQ,-0.3615292659832898),
 docplex.mp.LinearConstraint[input9](x_8,EQ,-0.6037614142464092)]

### OUTDATED!

> The keras_model predicted $C_1$ because:
>
> $x_0 = 2.967691214515491$,
>
> $x_3 = -1.408120229258977$,
>
> $x_5 = -0.790702170757714$, 
>
> $x_6 = 4.24127975754059$, 
>
> $x_7 = -0.3615292659832898$ and 
>
> $x_8 = -0.6037614142464092$.

### Trying to Improve the Explanation

Given a minimal explanation, can we improve it?

Constraints of type $x = c$ are equivalent to $x \le c \land x \ge c$.

Therefore, we need to substitute each $x = c$ constraint by the $x \le c$ and $x \ge c$ constraints.

Then, we try stretching the interval by substituting $x \le c$ by $x \le c + \Delta x$ and see if our prediction changes. If the prediction stays the same, then we substitue and try stretching it again. If the prediction changes, then this new interval isn't valid and we don't substitute. We found the upper bound of the interval, i.e. $x \le c$.

Then we try to stretch the interval to fin the lower bound. Analogously, We try substituting $c \ge x$ by ????/


We will end up with a pair of constraints the looks like $c - k_l \cdot \Delta{x} \le x$ and $x \le c + k_u \cdot \Delta{x}$, i.e. this pair represents $c - k_l \cdot \Delta{x} \le x \le c + k_u \cdot \Delta{x}$.

### Setting Up

In [8]:
import docplex

In [9]:
minimal_model = mdl_aux
testing_model = minimal_model.clone()

#### Quick Sratch

In [10]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input1](x_0,EQ,-0.2792306037275416),
 docplex.mp.LinearConstraint[input2](x_1,EQ,-0.267930022273471),
 docplex.mp.LinearConstraint[input3](x_2,EQ,0.6074173914551261),
 docplex.mp.LinearConstraint[input4](x_3,EQ,-0.8032068868488967),
 docplex.mp.LinearConstraint[input5](x_4,EQ,0.8170760979829164),
 docplex.mp.LinearConstraint[input6](x_5,EQ,0.0779031564307683),
 docplex.mp.LinearConstraint[input7](x_6,EQ,-0.4927117463954317),
 docplex.mp.LinearConstraint[input8](x_7,EQ,-0.3615292659832898),
 docplex.mp.LinearConstraint[input9](x_8,EQ,-0.6037614142464092)]

In [11]:
linear_constraints = testing_model.find_matching_linear_constraints('input')

for constraint in linear_constraints:
	testing_model.remove_constraint(constraint)
	testing_model.add_constraint(constraint.lhs <= constraint.rhs.clone(), 'input LE')
	testing_model.add_constraint(constraint.lhs >= constraint.rhs.clone(), 'input GE')

In [12]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input LE](x_0,LE,-0.2792306037275416),
 docplex.mp.LinearConstraint[input GE](x_0,GE,-0.2792306037275416),
 docplex.mp.LinearConstraint[input LE](x_1,LE,-0.267930022273471),
 docplex.mp.LinearConstraint[input GE](x_1,GE,-0.267930022273471),
 docplex.mp.LinearConstraint[input LE](x_2,LE,0.6074173914551261),
 docplex.mp.LinearConstraint[input GE](x_2,GE,0.6074173914551261),
 docplex.mp.LinearConstraint[input LE](x_3,LE,-0.8032068868488967),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-0.8032068868488967),
 docplex.mp.LinearConstraint[input LE](x_4,LE,0.8170760979829164),
 docplex.mp.LinearConstraint[input GE](x_4,GE,0.8170760979829164),
 docplex.mp.LinearConstraint[input LE](x_5,LE,0.0779031564307683),
 docplex.mp.LinearConstraint[input GE](x_5,GE,0.0779031564307683),
 docplex.mp.LinearConstraint[input LE](x_6,LE,-0.4927117463954317),
 docplex.mp.LinearConstraint[input GE](x_6,GE,-0.4927117463954317),
 docplex.mp.LinearConstraint[input LE](x_7,LE,-0.3615292

In [13]:
def get_x_ge_value_constraints(linear_constraints):
	# x >= c
	ge_constraints = []
	for xp in linear_constraints:
		if xp.sense == docplex.mp.constants.ComparisonType.GE:
			ge_constraints.append(xp)
	return ge_constraints

In [14]:
def get_x_le_value_constraints(linear_constraints):
	# x <= c
	le_constraints = []
	for xp in linear_constraints:
		if xp.sense == docplex.mp.constants.ComparisonType.LE:
			le_constraints.append(xp)
	return le_constraints

In [ ]:
epsilon = 0.01

variables = testing_model.find_matching_vars('x')

print('Initial constraints:')
print(get_x_ge_value_constraints(linear_constraints))
print(get_x_le_value_constraints(linear_constraints))

for constraint in linear_constraints:
	variable = constraint.lhs
	testing_model.solve()
	while testing_model.solution is None:
		# delta = variable.up
		if constraint.sense == docplex.mp.constants.ComparisonType.LE:
			# x <= c
			# Try x <= c + e
			pass

print()
print('Final constraints:')
print(get_x_ge_value_constraints(linear_constraints))
print(get_x_le_value_constraints(linear_constraints))

In [ ]:
# constraints = []
# for c in constraints:
# 	pass
m = docplex.mp.model.Model()
c = None
x = c.lhs

e0 = c.rhs.constant
e1 = x.ub
c.rhs = e1
m.solve()
if m.solution is None:  # i.e. it works
    pass

**TODO:** Rewrite pair of expression of type $x \le c$ and $x \ge c$ to $x = c$:

In [16]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

[docplex.mp.LinearConstraint[input LE](x_0,LE,5.127691214515445),
 docplex.mp.LinearConstraint[input GE](x_0,GE,2.967691214515491),
 docplex.mp.LinearConstraint[input LE](x_3,LE,-1.0781202292589767),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-1.458120229258977),
 docplex.mp.LinearConstraint[input LE](x_5,LE,-0.4507021707577137),
 docplex.mp.LinearConstraint[input GE](x_5,GE,-0.790702170757714),
 docplex.mp.LinearConstraint[input LE](x_6,LE,4.571279757540583),
 docplex.mp.LinearConstraint[input GE](x_6,GE,4.2112797575405905),
 docplex.mp.LinearConstraint[input LE](x_7,LE,-0.031529265983289645),
 docplex.mp.LinearConstraint[input GE](x_7,GE,-0.3615292659832898),
 docplex.mp.LinearConstraint[input LE](x_8,LE,-0.2837614142464089),
 docplex.mp.LinearConstraint[input GE](x_8,GE,-0.6037614142464092)]

Note that `x_6` is actually equal to `4.24127975754059`

In [17]:
number_of_inputs = len(dataframe.columns.drop('target'))
for i in range(number_of_inputs):
	constraints_of_x_i = filter(lambda x: x.lhs.name == f'x_{i}', linear_constraints)
	constraints = [c for c in constraints_of_x_i]

	if len(constraints) == 2:
		if constraints[0].rhs.constant == constraints[1].rhs.constant:
			testing_model.remove_constraints(constraints)
			testing_model.add_constraint(constraints[0].lhs == constraints[0].rhs, 'input')

In [18]:
improved_explanation = testing_model.find_matching_linear_constraints('input')
improved_explanation

[docplex.mp.LinearConstraint[input LE](x_0,LE,5.127691214515445),
 docplex.mp.LinearConstraint[input GE](x_0,GE,2.967691214515491),
 docplex.mp.LinearConstraint[input LE](x_3,LE,-1.0781202292589767),
 docplex.mp.LinearConstraint[input GE](x_3,GE,-1.458120229258977),
 docplex.mp.LinearConstraint[input LE](x_5,LE,-0.4507021707577137),
 docplex.mp.LinearConstraint[input GE](x_5,GE,-0.790702170757714),
 docplex.mp.LinearConstraint[input LE](x_6,LE,4.571279757540583),
 docplex.mp.LinearConstraint[input GE](x_6,GE,4.2112797575405905),
 docplex.mp.LinearConstraint[input LE](x_7,LE,-0.031529265983289645),
 docplex.mp.LinearConstraint[input GE](x_7,GE,-0.3615292659832898),
 docplex.mp.LinearConstraint[input LE](x_8,LE,-0.2837614142464089),
 docplex.mp.LinearConstraint[input GE](x_8,GE,-0.6037614142464092)]

### Pretty Printing the Explanation

In [19]:
def get_variable_index(variable: docplex.mp.dvar.Var) -> int:
	index = variable.name.split('_')[1]
	return int(index)

In [20]:
def print_explanation(explanation: list[docplex.mp.constr.LinearConstraint]):
	for e in explanation:
		variable = e.lhs
		index = get_variable_index(variable)
		feature_name = dataframe.columns[index]
		print(feature_name, e.sense.operator_symbol, e.rhs)

In [21]:
print_explanation(improved_explanation)

RI <= 5.127691214515445
RI >= 2.967691214515491
Al <= -1.0781202292589767
Al >= -1.458120229258977
K <= -0.4507021707577137
K >= -0.790702170757714
Ca <= 4.571279757540583
Ca >= 4.2112797575405905
Ba <= -0.031529265983289645
Ba >= -0.3615292659832898
Fe <= -0.2837614142464089
Fe >= -0.6037614142464092


## Comparing with Anchor

In [22]:
from anchor import utils

### Loading the Dataset

In [23]:
d = utils.load_csv_dataset(
	data=f'datasets/{dataset_name}/test.csv',
	target_idx=-1,
	feature_names=['RI','Na','Mg','Al','Si','K','Ca','Ba','Fe','target'],
	# categorical_features=None,
	# features_to_use=None,
	# feature_transformations=None,
	# discretize=False,
	# balance=False,
	# fill_na='-1',
	# filter_fn=None,
	skip_first=True
)

### Explainer

In [24]:
from anchor import anchor_tabular

In [25]:
explainer = anchor_tabular.AnchorTabularExplainer(
	d.class_names,
	d.feature_names,
	d.train,
	d.categorical_names
)

In [26]:
predict_fn = lambda x: tf.argmax(keras_model.predict(x)[0]).numpy().reshape(1)

In [27]:
for a in d.train:
	a == data[i]

/tmp/ipykernel_23061/1890216175.py:2: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  a == data[i]


In [28]:
exp = explainer.explain_instance(data[i, :-1], predict_fn)

1/1 [==============================] - 0s 29ms/step


In [29]:
exp.names()

['RI > -0.64', 'K > -0.60', 'Na <= 0.84', 'Si > -0.81', 'Al > -0.57']

In [30]:
print_explanation(improved_explanation)

RI <= 5.127691214515445
RI >= 2.967691214515491
Al <= -1.0781202292589767
Al >= -1.458120229258977
K <= -0.4507021707577137
K >= -0.790702170757714
Ca <= 4.571279757540583
Ca >= 4.2112797575405905
Ba <= -0.031529265983289645
Ba >= -0.3615292659832898
Fe <= -0.2837614142464089
Fe >= -0.6037614142464092


In [31]:
exp.precision()

0.2526837324525186

In [32]:
exp.coverage()

0.3867

In [33]:
# explainer.explain_instance(
# 	data_row,
# 	classifier_fn,
# 	threshold=0.95,
# 	delta=0.1,
# 	tau=0.15,
# 	batch_size=100,
# 	max_anchor_size=None,
# 	desired_label=None,
# 	**kwargs)